In [9]:
import sys
import os
import json

from IPython.display import display, HTML, clear_output
from ipywidgets import Dropdown, Text, Checkbox, Layout, Button, HBox, Output

# Add the src directory to the path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from genaisys import init_openai_api, make_openai_api_call
from genaisys.querying_functions import get_query_result, display_results

# ============ SHARED STATE ============
user_histories = {"User01": [], "User02": [], "User03": []}
active_user = "User01"
conversation_active = True

# Create output widgets
conversation_output = Output()
debug_output = Output()
user_memory = True

# ============ FUNCTIONS ============

def chat_with_gpt(messages, user_message):
    """Call GPT with optional RAG from Pinecone."""
    try:
        namespace = ""
        if "Pinecone" in user_message or "RAG" in user_message:
            if "Pinecone" in user_message:
                namespace = "genaisys"
            elif "RAG" in user_message:
                namespace = "data01"

            with debug_output:
                print(f"DEBUG: Using namespace: {namespace}")

            query_results = get_query_result(user_message, namespace)
            qtext, target_id = display_results(query_results)

            sc_input = qtext + " " + user_message
            task_response = make_openai_api_call(
                sc_input, "system",
                "You are an assistant who executes the tasks you are asked to do.",
                "user"
            )
            return f"{namespace}:{task_response}"
        else:
            if user_memory:
                user_messages_content = [
                    msg["content"] for msg in messages
                    if msg["role"] == "user" and "content" in msg
                ]
                combined = " ".join(user_messages_content)
                umessage = f"{combined} {user_message}"
            else:
                umessage = user_message

            task_response = make_openai_api_call(
                umessage, "system",
                "You are an assistant who executes the tasks you are asked to do.",
                "user"
            )
            return task_response
    except Exception as e:
        return f"An error occurred: {e}"


def save_conversation_history():
    """Save conversation history to JSON file."""
    filename = "conversation_history.json"
    with open(filename, 'w') as file:
        json.dump(user_histories, file, indent=4)
    display(HTML(f"<div style='color: green;'><strong>Saved to {filename}</strong></div>"))


def update_display():
    """Update the conversation display."""
    global active_user
    with debug_output:
        print(f"DEBUG: update_display() - Active user: {active_user}")
        print(f"DEBUG: History entries: {len(user_histories[active_user])}")

    with conversation_output:
        clear_output(wait=True)
        for entry in user_histories[active_user]:
            if entry['role'] == 'user':
                display(HTML(
                    f"<div style='margin-left: 20px; color: blue; margin-bottom: 5px;'>"
                    f"<strong>{active_user}:</strong> {entry['content']}</div>"
                ))
            elif entry['role'] == 'assistant':
                display(HTML(
                    f"<div style='margin-left: 20px; color: green; margin-bottom: 5px;'>"
                    f"<strong>Agent:</strong> {entry['content']}</div>"
                ))


def chat(user_message):
    """Handle user input and generate response."""
    global active_user, conversation_active

    with debug_output:
        print(f"DEBUG: chat() called with: '{user_message}'")

    # Check for exit
    if user_message.lower() in ['exit', 'quit']:
        conversation_active = False
        with conversation_output:
            clear_output()
            display(HTML("<div style='color: red;'><strong>Conversation ended.</strong></div>"))
            save_conversation_history()
        return

    # Add user message to history
    user_histories[active_user].append({"role": "user", "content": user_message})

    # Generate response if agent is enabled
    if agent_checkbox.value:
        with debug_output:
            print("DEBUG: Generating agent response...")
        try:
            response = chat_with_gpt(user_histories[active_user], user_message)
            with debug_output:
                print(f"DEBUG: Response: '{response[:100]}...'")
            user_histories[active_user].append({"role": "assistant", "content": response})
        except Exception as e:
            with debug_output:
                print(f"DEBUG: Error: {e}")
            user_histories[active_user].append({"role": "assistant", "content": f"Error: {e}"})
    else:
        with debug_output:
            print("DEBUG: Agent disabled, skipping response")

    update_display()


def on_user_change(change):
    """Handle user selection change."""
    global active_user
    active_user = change['new']
    with debug_output:
        print(f"DEBUG: User changed to: {active_user}")
    update_display()


def handle_input(sender):
    """Handle input submission (Enter key or button)."""
    user_message = input_box.value
    with debug_output:
        print(f"DEBUG: Input received: '{user_message}'")

    if user_message and user_message.strip():
        input_box.value = ""  # Clear input
        chat(user_message)

# Function to save conversation history to a file
def save_conversation_history():
    filename = "conversation_history.json"  # Define the filename
    with open(filename, 'w') as file:
        json.dump(user_histories, file, indent=4)  # Write the user histories dictionary to the file in JSON format
    display(HTML(f"<div style='color: green;'><strong>Conversation history saved to {filename}.</strong></div>"))

# ============ UI WIDGETS ============

user_selector = Dropdown(
    options=["User01", "User02", "User03"],
    value=active_user,
    description='User:',
    layout=Layout(width='200px')
)
user_selector.observe(on_user_change, names='value')

input_box = Text(
    placeholder="Type your message here and press Enter or click Send",
    layout=Layout(width='400px'),
    continuous_update=False
)
input_box.observe(lambda change: handle_input(change) if change['name'] == 'value' and change['new'] else None, names='value')

send_button = Button(
    description='Send',
    button_style='primary',
    layout=Layout(width='80px')
)
send_button.on_click(lambda b: handle_input(b))

agent_checkbox = Checkbox(
    value=True,
    description='Agent',
    layout=Layout(width='100px')
)

# ============ DISPLAY INTERFACE ============

display(HTML("<h3>GenAI Chat Interface</h3>"))
display(user_selector)
display(HBox([input_box, send_button]))
display(agent_checkbox)
display(HTML("<hr><h4>Conversation:</h4>"))
display(conversation_output)
display(HTML("<hr><h4>Debug Output:</h4>"))
display(debug_output)

with debug_output:
    print("DEBUG: Interface ready.")

Dropdown(description='User:', layout=Layout(width='200px'), options=('User01', 'User02', 'User03'), value='Use…

Checkbox(value=True, description='Agent', layout=Layout(width='100px'))

Output()

Output()